# Random Forest

Los bosques aleatorios tienen la particularidad de: **al entrenar un árbol, la búsqueda de la mejor división se realiza solo en un subconjunto de las características originales tomadas al azar.**

- Los subconjuntos aleatorios son diferentes para cada nodo dividido.
- El objetivo es inyectar aleatorización adicional en el procedimiento de aprendizaje para tratar de decorrelacionar los errores de predicción de los árboles individuales.

Por lo tanto, los bosques aleatorios están utilizando aleatorización en ambos ejes de la matriz de datos:
- Al generar muestras de arranque (boostrapping) para cada árbol en el bosque;
- Seleccionando aleatoriamente un subconjunto de características en cada nodo del árbol.

# Una mirada a los bosques aleatorios

In [2]:
# veremos el uso de un clasificador de bosque aleatorio en el conjunto de datos del censo de adultos.

import pandas as pd

adult_census = pd.read_csv("../../../data/adult-census-numeric/full.csv")
target_name = "class"
data = adult_census.drop(columns=[target_name, "education-num"])
target = adult_census[target_name]

> El censo de adultos contiene algunos datos categóricos. Los codificamos utilizando OrdinalEncoder, ya que los modelos basados ​​en árboles pueden funcionar de manera muy eficiente con una representación así de variables categóricas.

In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer, make_column_selector

categorical_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = make_column_transformer(
    (categorical_encoder, make_column_selector(dtype_include=object)),
    remainder="passthrough"
)

In [4]:
# Primero daremos un ejemplo simple en el que capacitaremos a un solo clasificador de árbol de decisión 
# y verificaremos su rendimiento de generalización a través de la validación cruzada.

from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

tree = make_pipeline(preprocessor, DecisionTreeClassifier(random_state=0))

In [5]:
from sklearn.model_selection import cross_val_score

scores_tree = cross_val_score(tree, data, target)

print(f"Decision tree classifier: "
      f"{scores_tree.mean():.3f} ± {scores_tree.std():.3f}")

Decision tree classifier: 0.820 ± 0.006


In [7]:
# Construimos un BaggingClassifier con un clasificador de árbol de decisión como modelo base.

from sklearn.ensemble import BaggingClassifier

bagged_trees = make_pipeline(
    preprocessor,
    BaggingClassifier(
        estimator=DecisionTreeClassifier(random_state=0),
        n_estimators=50, n_jobs=2, random_state=0,
    )
)

In [8]:
scores_bagged_trees = cross_val_score(bagged_trees, data, target)

print(f"Bagged decision tree classifier: "
      f"{scores_bagged_trees.mean():.3f} ± {scores_bagged_trees.std():.3f}")

Bagged decision tree classifier: 0.846 ± 0.005


Ahora, usaremos un **bosque aleatorio**.
> Observar que no necesitamos especificar ningún base_estimator porque el estimador se ve obligado a ser un árbol de decisión.
- Solo especificamos el número deseado de árboles en el bosque.


In [9]:
from sklearn.ensemble import RandomForestClassifier

random_forest = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators=50, n_jobs=2, random_state=0)
)

In [11]:
scores_random_forest = cross_val_score(random_forest, data, target)

print(f"Random forest classifier: "
      f"{scores_random_forest.mean():.3f} ± "
      f"{scores_random_forest.std():.3f}")

Random forest classifier: 0.851 ± 0.004


# Detalles sobre hiperparámetros predeterminados
Para los bosques aleatorios, es posible controlar la cantidad de aleatoriedad para cada división estableciendo el valor del hiperparámetro `max_features`:
- max_features=0.5 significa que el 50% de las características se deben considerar en cada división;
- max_features=1.0 significa que todas las características se deben considerar en cada división, lo que desactiva efectivamente el submuestreo de características.

De manera predeterminada, `RandomForestRegressor` deshabilita el submuestreo de características, mientras que `RandomForestClassifier` usa `max_features = np.sqrt(n_features)`.
- Estos valores predeterminados reflejan buenas prácticas dadas en la literatura científica.

> Sin embargo, max_Feature es uno de los **hiperparámetros a considerar** al ajustar un bosque aleatorio:
>- **demasiada aleatoriedad** en los árboles puede conducir a modelos base subajustados y puede ser perjudicial para la agrupación en su conjunto,
>- **muy poca aleatoriedad** en los árboles conduce a una mayor correlación de los errores de predicción y, como resultado, reduce los beneficios del paso de promediado en términos de control de sobreajuste.

**En la siguiente tabla se resumen estos detalles:**
<table class="colwidths-auto table">
<thead>
<tr class="row-odd"><th class="head"><p>Ensemble model class</p></th>
<th class="head"><p>Base model class</p></th>
<th class="head"><p>Default value for <code class="docutils literal notranslate"><span class="pre">max_features</span></code></p></th>
<th class="head"><p>Features subsampling strategy</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">BaggingClassifier</span></code></p></td>
<td><p>User specified (flexible)</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">n_features</span></code> (no&nbsp;subsampling)</p></td>
<td><p>Model level</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">RandomForestClassifier</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">DecisionTreeClassifier</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">sqrt(n_features)</span></code></p></td>
<td><p>Tree node level</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">BaggingRegressor</span></code></p></td>
<td><p>User specified (flexible)</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">n_features</span></code> (no&nbsp;subsampling)</p></td>
<td><p>Model level</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">RandomForestRegressor</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">DecisionTreeRegressor</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">n_features</span></code> (no&nbsp;subsampling)</p></td>
<td><p>Tree node level</p></td>
</tr>
</tbody>
</table>